##  Mapas de Calor da Biblioteca de Metassuperfícies – Library Heatmaps

---

### **PT - Resumo**
Este notebook demonstra a geração e interpretação de **mapas de calor** (heatmaps) para visualizar a cobertura de fase e amplitude da biblioteca de nanoestruturas. Os heatmaps mostram como as propriedades ópticas (fase e amplitude TE/TM) variam em função das dimensões geométricas (L_x, L_y).

Esta visualização é essencial para:
- **Avaliar a cobertura** da biblioteca no espaço de fases
- **Identificar lacunas** onde não há nanoestruturas caracterizadas
- **Planejar fabricação** de geometrias adicionais se necessário
- **Validar interpolação** usada no casamento de fase

<details>
<summary><b>Show English</b></summary>

### **EN - Summary**
This notebook demonstrates the generation and interpretation of **heatmaps** to visualize phase and amplitude coverage of the nanostructure library. Heatmaps show how optical properties (TE/TM phase and amplitude) vary as a function of geometric dimensions (L_x, L_y).

This visualization is essential for:
- **Assessing coverage** of the library in phase space
- **Identifying gaps** where no nanostructures are characterized
- **Planning fabrication** of additional geometries if needed
- **Validating interpolation** used in phase matching

</details>

---

### 1. 🔧 Imports

In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# Add src to path
sys.path.insert(0, str(Path().resolve().parent.parent / 'src'))

from meta_library import phase_matching

# Configurar matplotlib
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 10

### 2. ⚙️ Parâmetros | Parameters

---

#### **PT - Descrição**
Os parâmetros abaixo controlam a geração dos heatmaps:

| Parâmetro | Significado | Exemplo |
|------------|-------------|---------||
| `library_path` | Caminho para biblioteca limpa (CSV/Parquet) | `"library_cleaned.csv"` |
| `x_col`, `y_col` | Colunas para eixos X e Y do heatmap | `"L_x"`, `"L_y"` |
| `fields` | Campos a visualizar | `["phase_TE", "amp_TE", "phase_TM", "amp_TM"]` |
| `bins_x`, `bins_y` | Resolução da grade interpolada | `100`, `100` |
| `colormap` | Mapa de cores matplotlib | `"twilight"` (fases), `"viridis"` (amplitudes) |

Os heatmaps são gerados por interpolação dos dados discretos da biblioteca em uma grade regular.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
The parameters below control heatmap generation:

| Parameter | Meaning | Example |
|------------|----------|---------||
| `library_path` | Path to cleaned library (CSV/Parquet) | `"library_cleaned.csv"` |
| `x_col`, `y_col` | Columns for X and Y axes of heatmap | `"L_x"`, `"L_y"` |
| `fields` | Fields to visualize | `["phase_TE", "amp_TE", "phase_TM", "amp_TM"]` |
| `bins_x`, `bins_y` | Resolution of interpolated grid | `100`, `100` |
| `colormap` | Matplotlib colormap | `"twilight"` (phases), `"viridis"` (amplitudes) |

Heatmaps are generated by interpolating discrete library data onto a regular grid.

</details>

---

In [ ]:
# Parâmetros de entrada
library_path = "../../data/meta_library/library_cleaned.csv"  # Processado de Bibliotecas/Altura_Varia

# Configuração dos heatmaps
x_col = "L_x"
y_col = "L_y"
fields = ["phase_TE", "amp_TE", "phase_TM", "amp_TM"]
bins_x = 100
bins_y = 100

# Saída
run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_dir = Path(f"../../results/meta_library/heatmaps/demo_{run_id}")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Resultados serão salvos em: {output_dir}")

### 3. 📚 Teoria – Mapas de Calor | Theory – Heatmaps

---

#### **PT - Descrição**
Os heatmaps representam a resposta óptica das nanoestruturas como função de suas dimensões geométricas. Para cada ponto (L_x, L_y) no espaço de parâmetros, o heatmap mostra:

- **Fase TE/TM** (φ): Atraso de fase introduzido pela nanoestrutura [0, 2π rad]
- **Amplitude TE/TM** (|T|): Eficiência de transmissão [0, 1]

A interpolação é feita usando **griddata** com método linear/cubic, permitindo estimar valores em pontos não caracterizados experimentalmente. Áreas com dados esparsos podem apresentar artefatos de interpolação.

**Aplicação**: Os heatmaps permitem identificar rapidamente quais combinações (L_x, L_y) fornecem as fases/amplitudes desejadas para uma aplicação específica.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
Heatmaps represent the optical response of nanostructures as a function of their geometric dimensions. For each point (L_x, L_y) in parameter space, the heatmap shows:

- **TE/TM Phase** (φ): Phase delay introduced by the nanostructure [0, 2π rad]
- **TE/TM Amplitude** (|T|): Transmission efficiency [0, 1]

Interpolation is done using **griddata** with linear/cubic method, allowing estimation of values at points not experimentally characterized. Areas with sparse data may show interpolation artifacts.

**Application**: Heatmaps allow quick identification of which (L_x, L_y) combinations provide desired phases/amplitudes for a specific application.

</details>

---

### 3.5. 🔄 Processamento de Dados Brutos | Raw Data Processing

---

#### **PT - Descrição**
Antes de usar a biblioteca processada, é importante entender como os dados brutos são transformados. Os arquivos `.ts` (Touchstone) contêm parâmetros S em formato complexo (parte real e imaginária).

**Parâmetros S relevantes:**
- **S13**: Transmissão para polarização **TE** (Transverse Electric)
- **S24**: Transmissão para polarização **TM** (Transverse Magnetic)

**Transformações aplicadas:**

Para cada polarização (TE/TM), calculamos:

1. **Fase** (φ): `phase = arctan2(imag, real)`
   - Retorna fase em radianos no intervalo [-π, π]
   - Representa o atraso de fase introduzido pela nanoestrutura

2. **Amplitude** (|T|): `amp = sqrt(real² + imag²)`
   - Magnitude do coeficiente de transmissão
   - Valores próximos de 1 indicam alta eficiência

**Exemplo de processamento:**
```python
# Para TE (usando S13)
df['phase_TE'] = np.arctan2(df['S13_imag'], df['S13_real'])
df['amp_TE'] = np.sqrt(df['S13_real']**2 + df['S13_imag']**2)

# Para TM (usando S24)
df['phase_TM'] = np.arctan2(df['S24_imag'], df['S24_real'])
df['amp_TM'] = np.sqrt(df['S24_real']**2 + df['S24_imag']**2)
```

**Nota**: A biblioteca usada neste notebook já está processada. Para processar novos dados brutos, use o módulo `src/meta_library/clean_library.py` ou `src/meta_library/generate_df.py`.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
Before using the processed library, it's important to understand how raw data is transformed. The `.ts` (Touchstone) files contain S-parameters in complex format (real and imaginary parts).

**Relevant S-parameters:**
- **S13**: Transmission for **TE** polarization (Transverse Electric)
- **S24**: Transmission for **TM** polarization (Transverse Magnetic)

**Applied transformations:**

For each polarization (TE/TM), we calculate:

1. **Phase** (φ): `phase = arctan2(imag, real)`
   - Returns phase in radians in the range [-π, π]
   - Represents the phase delay introduced by the nanostructure

2. **Amplitude** (|T|): `amp = sqrt(real² + imag²)`
   - Magnitude of the transmission coefficient
   - Values close to 1 indicate high efficiency

**Processing example:**
```python
# For TE (using S13)
df['phase_TE'] = np.arctan2(df['S13_imag'], df['S13_real'])
df['amp_TE'] = np.sqrt(df['S13_real']**2 + df['S13_imag']**2)

# For TM (using S24)
df['phase_TM'] = np.arctan2(df['S24_imag'], df['S24_real'])
df['amp_TM'] = np.sqrt(df['S24_real']**2 + df['S24_imag']**2)
```

**Note**: The library used in this notebook is already processed. To process new raw data, use the `src/meta_library/clean_library.py` or `src/meta_library/generate_df.py` modules.

</details>

---

### 4. 📂 Carregamento da Biblioteca | Loading the Library

---

#### **PT - Descrição**
Carregamos a biblioteca de nanoestruturas processada.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We load the processed nanostructure library.

</details>

---

In [ ]:
# Carregar biblioteca
df = pd.read_csv(library_path) if library_path.endswith('.csv') else pd.read_parquet(library_path)

print(f"📊 Biblioteca carregada: {len(df)} registros")
print(f"\n   Verificando colunas necessárias:")
required = [x_col, y_col] + fields
for col in required:
    status = '✓' if col in df.columns else '✗'
    print(f"   {status} {col}")

# Estatísticas básicas
print(f"\n   Ranges:")
print(f"   L_x: [{df['L_x'].min():.3f}, {df['L_x'].max():.3f}] μm")
print(f"   L_y: [{df['L_y'].min():.3f}, {df['L_y'].max():.3f}] μm")
if 'H' in df.columns:
    print(f"   H: {df['H'].unique()} μm")

# Visualizar distribuição de pontos
plt.figure(figsize=(8, 6))
plt.scatter(df['L_x'], df['L_y'], alpha=0.5, s=20)
plt.xlabel('L_x (μm)')
plt.ylabel('L_y (μm)')
plt.title(f'Distribuição de Nanoestruturas na Biblioteca (N={len(df)})')
plt.grid(alpha=0.3)
plt.savefig(output_dir / 'library_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

### 5. 🔥 Geração dos Heatmaps | Heatmap Generation

---

#### **PT - Descrição**
Geramos os heatmaps para todos os campos especificados usando a função `compute_heatmaps`.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We generate heatmaps for all specified fields using the `compute_heatmaps` function.

</details>

---

In [ ]:
# Gerar heatmaps
print("🔄 Gerando heatmaps...")
heatmaps = phase_matching.compute_heatmaps(
    df=df,
    x=x_col,
    y=y_col,
    fields=tuple(fields),
    bins_x=bins_x,
    bins_y=bins_y
)

print(f"✓ Heatmaps gerados!")
for field, hmap in heatmaps.items():
    print(f"   {field}: shape={hmap.shape}, range=[{hmap.min():.3f}, {hmap.max():.3f}]")

### 6. 📊 Visualização dos Heatmaps | Heatmap Visualization

---

#### **PT - Descrição**
Visualizamos todos os heatmaps gerados em uma grade 2x2.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We visualize all generated heatmaps in a 2x2 grid.

</details>

---

In [ ]:
# Visualizar heatmaps
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

# Extent para imshow
extent = [df['L_x'].min(), df['L_x'].max(), df['L_y'].min(), df['L_y'].max()]

for idx, field in enumerate(fields):
    ax = axes[idx]
    hmap = heatmaps[field]
    
    # Escolher colormap apropriado
    if 'phase' in field.lower():
        cmap = 'twilight'
        vmin, vmax = 0, 2*np.pi
    else:
        cmap = 'viridis'
        vmin, vmax = 0, 1
    
    im = ax.imshow(hmap.T, origin='lower', extent=extent, cmap=cmap, 
                   vmin=vmin, vmax=vmax, aspect='auto')
    ax.set_xlabel('L_x (μm)')
    ax.set_ylabel('L_y (μm)')
    ax.set_title(field)
    plt.colorbar(im, ax=ax, label='rad' if 'phase' in field.lower() else 'Transmissão')

plt.tight_layout()
plt.savefig(output_dir / 'heatmaps_all.png', dpi=150, bbox_inches='tight')
plt.show()

### 7. 🔍 Análise da Cobertura | Coverage Analysis

---

#### **PT - Descrição**
Analisamos a cobertura de fase alcançada pela biblioteca.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We analyze the phase coverage achieved by the library.

</details>

---

In [ ]:
# Análise de cobertura de fase
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histograma fase TE
axes[0].hist(df['phase_TE'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axhline(y=len(df)/(2*np.pi), color='r', linestyle='--', label='Cobertura uniforme ideal')
axes[0].set_xlabel('Fase TE (rad)')
axes[0].set_ylabel('Frequência')
axes[0].set_title('Distribuição de Fase TE')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Histograma fase TM
axes[1].hist(df['phase_TM'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].axhline(y=len(df)/(2*np.pi), color='r', linestyle='--', label='Cobertura uniforme ideal')
axes[1].set_xlabel('Fase TM (rad)')
axes[1].set_ylabel('Frequência')
axes[1].set_title('Distribuição de Fase TM')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'phase_coverage_histograms.png', dpi=150, bbox_inches='tight')
plt.show()

# Estatísticas
print("\n📈 Cobertura de Fase:")
print(f"   TE: [{df['phase_TE'].min():.3f}, {df['phase_TE'].max():.3f}] rad")
print(f"   TM: [{df['phase_TM'].min():.3f}, {df['phase_TM'].max():.3f}] rad")
print(f"\n   Cobertura TE: {(df['phase_TE'].max() - df['phase_TE'].min())/(2*np.pi)*100:.1f}% de 2π")
print(f"   Cobertura TM: {(df['phase_TM'].max() - df['phase_TM'].min())/(2*np.pi)*100:.1f}% de 2π")

### 8. 💾 Salvamento dos Heatmaps | Saving Heatmaps

---

#### **PT - Descrição**
Salvamos os heatmaps como arrays NumPy para uso posterior.

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
We save heatmaps as NumPy arrays for later use.

</details>

---

In [ ]:
# Salvar heatmaps
for field, hmap in heatmaps.items():
    np.save(output_dir / f'heatmap_{field}.npy', hmap)

print(f"✓ Heatmaps salvos em: {output_dir}")
for field in fields:
    print(f"   - heatmap_{field}.npy")
print(f"\n   Figuras:")
print(f"   - library_distribution.png")
print(f"   - heatmaps_all.png")
print(f"   - phase_coverage_histograms.png")

### 9. 🔄 Reprodutibilidade | Reproducibility

---

#### **PT - Descrição**
Para reproduzir estes resultados:

1. **Biblioteca**: Use a mesma biblioteca limpa (mesmo arquivo CSV/Parquet)
2. **Parâmetros**: Use os mesmos valores de `bins_x`, `bins_y`, `fields`
3. **Versão**: Código testado com Python 3.10+ e dependências do `requirements.txt`

**Comando CLI equivalente**:
```bash
python src/cli/run_heatmaps.py \
  --library ../../data/meta_library/library_cleaned.csv \
  --out-dir ../../results/meta_library/heatmaps/run_<timestamp> \
  --fields phase_TE amp_TE phase_TM amp_TM \
  --bins-x 100 --bins-y 100 \
  --colormap twilight
```

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
To reproduce these results:

1. **Library**: Use the same cleaned library (same CSV/Parquet file)
2. **Parameters**: Use the same values for `bins_x`, `bins_y`, `fields`
3. **Version**: Code tested with Python 3.10+ and dependencies from `requirements.txt`

**Equivalent CLI command**:
```bash
python src/cli/run_heatmaps.py \
  --library ../../data/meta_library/library_cleaned.csv \
  --out-dir ../../results/meta_library/heatmaps/run_<timestamp> \
  --fields phase_TE amp_TE phase_TM amp_TM \
  --bins-x 100 --bins-y 100 \
  --colormap twilight
```

</details>

---

### 10. 💡 Considerações Finais | Final Considerations

---

#### **PT - Descrição**
Este notebook demonstrou a geração de heatmaps para bibliotecas de metassuperfícies. Pontos importantes:

- **Cobertura de fase**: Idealmente deve cobrir [0, 2π] para ambas polarizações
- **Densidade de pontos**: Maior densidade → melhor interpolação → menor erro no casamento
- **Lacunas**: Áreas com poucos dados podem gerar artefatos visuais e erros de casamento
- **Transmissão**: Valores próximos a 1 indicam alta eficiência

**Uso dos heatmaps**:
1. **Planejamento**: Identificar geometrias faltantes antes de simulações/fabricação
2. **Validação**: Verificar se a biblioteca cobre o espaço de fases necessário
3. **Otimização**: Escolher regiões (L_x, L_y) com melhor trade-off fase/amplitude
4. **Documentação**: Visualizar facilmente as capacidades da biblioteca

<details>
<summary><b>Show English</b></summary>

#### **EN - Description**
This notebook demonstrated heatmap generation for metasurface libraries. Important points:

- **Phase coverage**: Ideally should cover [0, 2π] for both polarizations
- **Point density**: Higher density → better interpolation → lower matching error
- **Gaps**: Areas with sparse data can generate visual artifacts and matching errors
- **Transmission**: Values close to 1 indicate high efficiency

**Heatmap uses**:
1. **Planning**: Identify missing geometries before simulations/fabrication
2. **Validation**: Verify if library covers required phase space
3. **Optimization**: Choose (L_x, L_y) regions with best phase/amplitude trade-off
4. **Documentation**: Easily visualize library capabilities

</details>

---